[Here are the slides](https://docs.google.com/presentation/d/11uyJMpCyVceYlC50QUdVp3DJe42BLxW4CHykxIdR7AU/edit?usp=sharing)


# Opening Images in Python

How to use the Pillow library

In [2]:
from PIL import Image
# PIL = python image library

img = Image.open("alice.png")
img.show()

# Asking User to Verify a PNG Signature 

Requesting user input. Does the supplied signature photo make sense?

In [4]:
def validate():
    user_input = input("Is this a valid signature? (y/n)")
    if user_input.lower():
        print("It's valid.")
    elif user_input.lower():
        print("It's invalid.")
    else:
        validate() # recursive if invalid input.

def display_and_validate(filename):
    img = Image.open(filename)
    img.show()
    validate()

In [5]:
display_and_validate("alice.png")

Is this a valid signature? (y/n)y
It's valid.


# Defining the Coin

Stupid-simple Python class

In [23]:
class PNGCoin:
    '''This represents transfers of ownership'''
    def __init__(self, transfers): # dunder = __x__
        self.transfers = transfers
    # __init_, initializing the class instance, takes self(the raw class object, here PNGCoin, and modifies it as per init)

# Validating a Coin

Here we check every entry in PNGCoin.transfers

In [24]:
coin = PNGCoin([
    Image.open("alice.png"),
    Image.open("alice-to-bob.png"),
])

In [ ]:
bad_coin = PNGCoin([
    Image.open("alice.png"),
    Image.open("alice-to-bob-forged.png"),
])

In [38]:
def handle_user_input(user_input):
    if user_input.lower() == 'y':
        return True
    elif user_input.lower() == 'n':
        return False
    else:
        user_input = input("Is this a valid signature? (y/n)") #since recursion, get the input again
        handle_user_input(user_input) # recursive if invalid input.

# broke the function into 2 since you don't want to run for loop everytime user gives invalid input.

def validate(coin):
    for transfer in coin.transfers:
        transfer.show()
        user_input = input("Is this a valid signature? (y/n)")
        is_valid = handle_user_input(user_input)
        if not is_valid:
            return False
    return True

In [42]:
validate(coin)

Is this a valid signature? (y/n)y
Is this a valid signature? (y/n)y


True

In [27]:
coin.transfers

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=651x797 at 0x7F7ECA0BC898>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=810x607 at 0x7F7ECA0BC978>]

In [28]:
coin.transfers[1].show()

# Serializing Coins 

Let's take the coin defined ^^ and write it to disk

In [44]:
'''Serialization is putting data in a format that is easily transportable - binary format.'''
import pickle

def serialize(coin):
    return pickle.dumps(coin)

In [45]:
def to_disk(coin, filename):
    '''Takes a coin and writes it to disk with the filename.'''
    serialized = serialize(coin)
    with open(filename, 'wb') as f: #with open is better as it opens the file in context manage and closes it  after finished working.
        f.write(serialized)

In [46]:
!ls | grep "bobs.pngcoin" #ls current dir & grep for the file > no result as doesn't exist.

In [47]:
to_disk(coin, "bobs.pngcoin") # we write the coin to 'bobs.pngcoin' file

In [48]:
!ls | grep "bobs.pngcoin" # we see the file/coin

bobs.pngcoin


# Deserializing Coins

Let's take the `bobs.pngcoin` file we created ^^ and read it back into Python

In [49]:
'''deserialize = convert byte data to human readable data'''

def deserialize(serialized):
    return pickle.loads(serialized)

In [51]:
# checking if deserialize func works
coin2 = deserialize(serialize(coin))
coin2.transfers == coin.transfers

True

In [60]:
def from_disk(filename):
    with open(filename, 'rb') as f:
        serialized = f.read()
        return deserialize(serialized) # deserialized version of the coin, a python object

In [61]:
coin3 = from_disk("bobs.pngcoin")

coin3.transfers == coin.transfers

True

# Using the Final Library

I also wrote a [pngcoin.py](pngcoin.py) library that uses a more object-oriented design. Let's explore how it works:

In [66]:
import pngcoin as pc

In [67]:
coin = pc.PNGCoin([
    Image.open("alice.png"),
    Image.open("alice-to-bob.png"),
])

In [70]:
coin.validate()

Is this a valid minting signature? (y/n)y
Is this a valid minting signature? (y/n)y


True

In [71]:
coin.to_disk("library-example.pngcoin") # takes self & filename = self is implicit

In [75]:
coin = pc.PNGCoin.from_disk("library-example.pngcoin") # takes cls & filename, cls??
coin

In [76]:
coin.validate()

Is this a valid minting signature? (y/n)y
Is this a valid minting signature? (y/n)y


True